<h1 align='center'  style = 'margin-bottom: 0px'> The Pencil _Shaft_ Factory </h1>
<h2 align='center'  style = 'margin-top: 0px'> The Graphical Method for <br/> Linear Programs <br/><i>Interactions Only</i> </h2>
<p>&nbsp;</p>

__Note__: This notebook contains only the _interactions_ for the Main Example (intended for use in lecture).  

We begin by __import__ing some Python modules we will use below:  Place your cursor in the cell below (i.e., select the cell), and then execute __[ctrl]-[enter]__.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import numpy as np 
from ipywidgets import interact

<p>&nbsp;</p>

## The Linear Program
Solving our main example means finding the solution(s) to the following linear program:  
$$ maximize \; z = 2x+3y \quad subject \; to \; (x,y) \in \Omega $$ 

where the feasible region $\Omega$ is given by 
$$ \Omega = \left\lbrace (x,y)\in\mathbb{R}^2 \; \; \middle\vert
      \begin{array}{c} 
        x + \; y  \le  20 \\ 
        x + 2y  \le 30 \\
        x  \le 15 \\ 
        y  \le 2x \\ 
        x \ge 0, \quad y \ge 0
      \end{array}
    \right\rbrace 
$$
Similar to the way in which we stated the problem, there are 2 tasks we must address

1. Determining the structure of the Feasible Region. 

2. Finding where the largest value of $z$ occurs over the Feasible Region. 

Let's now consider the role of the objective function (issue #2). 

In some sense, if we know all the vertices of the Feasible region, then we need only substitute them respectively into the objective and determine which produces the largest value of $z$.    This works great for small examples, but remember that real world applications involves tens, hundreds, thousands, and even millions of variables, and in such contexts, a feasible region can have arbitrarily huge numbers of vertices. 

<div align = "center" style = "font-size:larger; color:blue; margin-top:20px; margin-bottom:20px"> 
Our goal is a method that computes vertices only as long as the objective is increasing. </div>

This method will be called the _Simplex Method_, and its central feature is that by tracking the objective function we can arrive at a solution to a linear program via calculation of only a relatively few of the vertices of the feasible region. 

<p>&nbsp;</p>

## Optimizating an Objective over a Feasible Region 
First off, if $\Omega \subset \mathbb{R}^n$, then the objective as an equation introduces an additional variable $z$.  Moreover, since the objective is a linear function, the linear program is actually asking for a vertex with largest possible $z$ on a polytope in $n+1$ dimensional space.   

For example, when we write the objective function in equation form, such as in  
$$ maximize \; z = 2x+3y \quad instead\;of\quad maximize 2x+3y,$$
we are actually adding a new constraint.  Indeed, in our main example, the linear program 
\begin{eqnarray*} 
maximize   & z = 2x+3y \\
subject \; to & x + \; y  \le  20 \\ 
              & x + 2y  \le 30 \\
              & x  \le 15 \\ 
              & y  \le 2x \\ 
              & x\ge 0, \quad y \ge 0
\end{eqnarray*}
should -- and does -- have exactly the same solution as the linear program 
\begin{eqnarray*} 
maximize   & z  \\
subject \; to & z = 2x+3y  \\ 
              & x + \; y  \le  20 \\ 
              & x + 2y  \le 30 \\
              & x  \le 15 \\ 
              & y  \le 2x \\ 
              & x\ge 0, \quad y \ge 0
\end{eqnarray*}
However, this last linear program is in _3 dimensions_ (i.e., in the 3 independent variables $x,y,z$).  Indeed, we can write it in the form 
\begin{eqnarray*} 
maximize   & z  \\
subject \; to & 2x+3y    & - z & = 0  \\ 
              & x + \; y &     &  \le  20 \\ 
              & x + 2y   &     & \le 30 \\
              & x        &     & \le 15 \\ 
              & -2x + y  &     &  \le 0 \\ 
              & x\ge 0,  &     & y \ge 0
\end{eqnarray*}
The feasible region for this last program, which we will call $\Omega'$, is still a polygon, but in the plane $z = 2x+3y$ rather than in the xy-plane. The objective is to find the vertex of $\Omega'$ with the largest $z$ coordinate, which in our main example is $(10,10,50)$.   

All this is illustrated below: 

In [ ]:
vertices = [ (0,0), (15,0), (15,5), (10,10), (6,12), (0,0) ]
XY = np.array(vertices).T
X,Y = XY[0], XY[1]
Z = 2*X+3*Y

fig = plt.figure(figsize = (8,8))
ax3d = fig.gca(projection='3d')

ax3d.plot_trisurf(X, Y, Z, linewidth=1, antialiased=True, alpha = 0.5, color='cyan', shade = False)
ax3d.plot_trisurf(X, Y, 0*Z, antialiased = True, color = 'cyan', alpha = 0.5)
ax3d.plot_trisurf([0,17,17,0],[0,0,14,14], [0,34,76,42], antialiased = True, color = 'pink', alpha = 0.25, shade = False)

ax3d.set_xlabel("x"); ax3d.set_ylabel("y"); ax3d.set_zlabel("z")
for i in range(len(X)):
    ax3d.plot([X[i],X[i]],[Y[i],Y[i]],[0,Z[i]], color='darkblue', alpha = 0.5)
ax3d.plot(X,Y,   Z, color = 'blue')#, alpha = 0.5)
ax3d.plot(X,Y, 0*Z, color = 'darkblue', alpha = 0.5)
ax3d.scatter([10],[10],[50], color='red')
ax3d.text(5,5,25, "$\Omega'$", fontsize = 14)
ax3d.text(12,12,60, '$z = 2x+3y$', fontsize = 14, color = 'pink')
ax3d.text(10,10,50,'  $(10,10,50)$')
ax3d.text(10,4,0, '$\Omega$', fontsize = 14)
ax3d.set_xlim(0,16); ax3d.set_ylim(0,14);ax3d.set_zlim(0,55)
ax3d.set_title('$(x*,y*,z*) = (10,10,50)$\n', fontsize = 14)


Indeed, in general, if we write the objective in equation form $z = ...$, then we are actually adding a constraint and creating a space that is one dimension higher than the original.  This brings up a very important concept in operations research in general.  

<div align = "center" style="margin-top:20px;margin-bottom:20px;font-size:larger;color:blue"> Solving linear and nonlinear programs often means creating new, often 'artificial' variables <br/> and working in typically much higher dimensional spaces than the original problem.</div>

However, rather than think of a "new" feasible region, $\Omega'$, we instead simply think of the original feasible region $\Omega$ subjected to the objective function $2x+3y$, which maps $\Omega$ into the plane with equation $z = 2x+3y$. 

In [ ]:
vertices = [ (0,0), (15,0), (15,5), (10,10), (6,12), (0,0) ]
XY = np.array(vertices).T
X,Y = XY[0], XY[1]
Z = 2*X+3*Y

fig = plt.figure(figsize = (8,8))
ax3d = fig.gca(projection='3d')

def LPpolyhedron(ax):
    ax.plot_trisurf(X, Y, Z, linewidth=1, antialiased=True, alpha = 0.5, color='cyan', shade = False)
    ax.plot_trisurf(X, Y, 0*Z, antialiased = True, color = 'cyan', alpha = 0.5)
    ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")
    for i in range(len(X)):
        ax.plot([X[i],X[i]],[Y[i],Y[i]],[0,Z[i]], color='darkblue', alpha = 0.5)
    ax.plot(X,Y,   Z, color = 'blue')#, alpha = 0.5)
    ax.plot(X,Y, 0*Z, color = 'darkblue', alpha = 0.5)
    ax.scatter([10],[10],[50], color='red')
    ax.text(5,5,40, "$z = 2x+3y$", fontsize = 14)
    ax.text(10,10,50,'  $(10,10,50)$')
    ax.text(12,6,0, '$\Omega$', fontsize = 14)
    ax.set_xlim(0,16); ax.set_ylim(0,14);ax.set_zlim(0,55)
    ax.set_title('$(x*,y*,z*) = (10,10,50)$\n', fontsize = 14)
LPpolyhedron(ax3d)

The advantage of this latter conceptualization of a linear program is that it allows us to work directly with $\Omega$ in the xy-plane, and to think of the added dimensionality in terms of _level curves_ or _isolines_ of the objective rather than as a new object in a higher dimensional context.  

That is, it is better to think of the objective function as a _ __ family of level curves__ _.  For example, the objective 
$ z = 2x+3y$ can be considered to be the set of all lines of the form $$ 2x+3y= k \qquad and \; z = k.$$

_Notice that as z increases, the level curves move from vertex to vertex until a maximum is achieved._

In [ ]:
fig = plt.figure(figsize = (8,8))
ax3d = fig.gca(projection='3d')

LPpolyhedron(ax3d)

inPlane = None
onSurf = None
lbl1, lbl2 = None, None
def LevelCurve( k = 0 ):
    global inPlane, onSurf, lbl1, lbl2
    try:
        ax3d.lines.remove(inPlane[0])
        ax3d.lines.remove(onSurf[0])
        ax3d.texts.remove(lbl1)
        ax3d.texts.remove(lbl2)
    except:
        pass
    if( k < 30):
        xr, yr = k/2, 0
    else:  #( 30 <= k < 55 ):
        xr, yr = 15, k/3 - 10
        
    inPlane = ax3d.plot([k/8,xr],[k/4,yr], color='magenta')
    onSurf  = ax3d.plot([k/8,k/8,xr,xr],[k/4,k/4,yr,yr],[0,k,k,0], color='magenta', linestyle = 'dashed')
    lbl1    = ax3d.text( k/16 + xr/2, k/8 + yr/2, 0, '$2x+3y=%s$' % k, color='magenta')
    lbl2    = ax3d.text( k/8, k/4, k/2, '$z = %s$' % k, color='magenta')

interact(LevelCurve, k=(0,50,1));

Thus, we can replace working with $z=2x+3y$ in 3 dimensions by the family of level curves $$2x+3y=k, \; k\in\mathbb{R}.$$  This allows us to work with the objective function in terms of the feasible region, and in fact, _it will be our preferred and indeed only approach used for visualizing the objective function._ 

Below we have the same visualization as above but restricted to the xy-plane. 

In [ ]:
fig, axes = plt.subplots()

#plot the Feasible Region
vertices = [ (0,0), (15,0), (15,5), (10,10), (6,12) ]
X,Y = zip(*vertices)
axes.fill(X,Y, facecolor='cyan', edgecolor='black', zorder = 1)

#plot the vertices
axes.scatter(X,Y, color='brown', zorder = 2)
for vert in vertices:
    axes.text( vert[0], vert[1], " ({0},{1})".format(vert[0], vert[1]), fontsize = 14)

# Label the resulting feasible region
axes.text(5,3, '$\Omega$', fontsize = 14 )

# Plot the xy-axes and set upper bound on y
axes.set_xlim(0,30)
axes.set_ylim(0,20)
axes.set_xlabel("x = 1000's of Standard shafts\n")
axes.set_ylabel("y = 1000's of Engineering shafts")

Line, Label = None, None
def LevelCurve(k):
    global Line, Label
    try:
        axes.lines.remove(Line[0])
        axes.texts.remove(Label)
    except:
        pass
    
    Line  = axes.plot( [0,k/2], [k/3,0], color='magenta' )
    Label = axes.text( k/4, k/6, ' $2x+3y=%s$' % k, color='magenta', fontsize = 14)
    axes.set_title('$k=%s$' % k, fontsize = 14)
    axes.grid(True)

interact(LevelCurve, k=(0,80,1));

One immediate consequence is that we can use increasing values of $k$ to avoid computing many of the vertices.  For example, we know that $z=0$ at the origin, and as we increase $x$ from 0 to 15, the value of $k$ increases from 0 to 30 (_Try it out!!_). 

For $x=15$, we then increase $y$ from 0 to $5$, which increases $k$ from 30 to 45.  (_Try it out!_). 

We cannot continue, but must instead turn along the constraint $ x + y = 20$, along which $k$ increases from 45 to 50.  

However, if we then start to move along $x+2y = 30$, then $k$ _ __ decreases__ _ from 50 to 48.  

Conversely, as $k$ increases, the line $2x+3y=k$ leaves the feasible region. Thus, the answer is $z^* = 50$ corresponding to 
$$ argmax(z) = (10,10) $$
which is to say that $(x^*,y^*) = (10,10)$. 